# List OpenAI Conversation Messages

This notebook lists all messages/items in an OpenAI conversation using the Responses API.

Target conversation_id: `conv_693f891a597481979c3796904168941f0b9bf5b26aad7d61`


In [41]:
import yaml
from pathlib import Path
from openai import OpenAI
import json
from datetime import datetime

# Load OpenAI config from config files
config_path = Path.cwd().parent / "config" / "config.yaml"
secrets_path = Path.cwd().parent / "config" / "secrets.yaml"

with open(config_path, "r", encoding="utf-8") as f:
    config_values = yaml.safe_load(f)

with open(secrets_path, "r", encoding="utf-8") as f:
    secrets_values = yaml.safe_load(f)

# Merge config and secrets
openai_config = config_values["openai"] | secrets_values["openai"]

# Initialize OpenAI client
client = OpenAI(
    api_key=openai_config["api_key"],
    base_url=openai_config["base_url"]
)

print("✅ OpenAI client initialized")
print(f"Base URL: {openai_config['base_url']}")


✅ OpenAI client initialized
Base URL: https://api.openai.com/v1


In [42]:
# Target conversation ID
conversation_id = "conv_6941e3b17fa0819792ec2fb0efde3db6016d31c69598a7b7"

# Retrieve the conversation metadata first
conversation = client.conversations.retrieve(conversation_id)

print(f"✅ Retrieved conversation: {conversation_id}")
print(f"Conversation ID: {conversation.id}")
print(f"Created at: {getattr(conversation, 'created_at', 'N/A')}")
print(f"Metadata: {getattr(conversation, 'metadata', {})}")

# List all items using the items.list() API with pagination support
print(f"\n📋 Fetching items using conversations.items.list()...")
all_items = []
after = None
has_more = True
page = 1

while has_more:
    params = {"conversation_id": conversation_id}
    if after:
        params["after"] = after
    
    items_response = client.conversations.items.list(**params, order="asc")
    page_items = items_response.data if hasattr(items_response, 'data') else []
    all_items.extend(page_items)
    
    has_more = getattr(items_response, 'has_more', False)
    if page_items:
        after = page_items[-1].id if hasattr(page_items[-1], 'id') else None
    
    print(f"   Page {page}: Retrieved {len(page_items)} items (Total: {len(all_items)})")
    page += 1
    
    if not has_more:
        break

items = all_items
items_response_meta = items_response  # Keep last response for metadata

print(f"✅ Retrieved {len(items)} total items from conversation")


✅ Retrieved conversation: conv_6941e3b17fa0819792ec2fb0efde3db6016d31c69598a7b7
Conversation ID: conv_6941e3b17fa0819792ec2fb0efde3db6016d31c69598a7b7
Created at: 1765925809
Metadata: {'chat_id': '26517046-0', 'name': '张宇', 'job_applied': '架构师', 'mode': 'greet'}

📋 Fetching items using conversations.items.list()...
   Page 1: Retrieved 12 items (Total: 12)
✅ Retrieved 12 total items from conversation


In [43]:
# Display all items in the conversation
print(f"\n📋 Listing all {len(items)} items in conversation:\n")
print("=" * 80)

for idx, item in enumerate(items, 1):
    item_type = getattr(item, 'type', 'unknown')
    item_id = getattr(item, 'id', 'N/A')
    
    if item_type == 'message':
        role = getattr(item, 'role', 'unknown')
        content_raw = getattr(item, 'content', '')
        
        # Extract text from ResponseOutputText objects
        content_text = ""
        if isinstance(content_raw, list):
            for content_item in content_raw:
                if hasattr(content_item, 'text'):
                    content_text += content_item.text
                elif isinstance(content_item, str):
                    content_text += content_item
        elif isinstance(content_raw, str):
            content_text = content_raw
        
        print(f"[{idx}]{role}: {content_text[:30]}" + ("..." if len(content_text) > 30 else ""))
        
        # Try to parse JSON if it looks like JSON
        if content_text.strip().startswith('{') or content_text.strip().startswith('['):
            try:
                json_obj = json.loads(content_text)
                print(f"    ✅ Contains valid JSON: {list(json_obj.keys())}")
            except:
                pass
    else:
        # For non-message items, print all attributes
        print(f"[{idx}] Item Type: {item_type}")
        print(f"Attributes: {item}")
    
    print("-" * 80)



📋 Listing all 12 items in conversation:

[1]developer: 你是招聘顾问，正在和候选人沟通。你的目标是根据岗位要求分析简...
--------------------------------------------------------------------------------
[2]developer: 12月15日 沟通的职位-资深java开发工程师/架构师
--------------------------------------------------------------------------------
[3]user: 您好，可以聊聊吗？您这个职位我很有兴趣，希望进一步了解
--------------------------------------------------------------------------------
[4]user: 招聘顾问您好，你帮我分析一下我是否匹配架构师这个岗位？以下是...
--------------------------------------------------------------------------------
[5] Item Type: reasoning
Attributes: ResponseReasoningItem(id='rs_016d31c69598a7b7006941e3b341e4819796ee0c945abcdb63', summary=[], type='reasoning', content=None, encrypted_content=None, status=None)
--------------------------------------------------------------------------------
[6]assistant: {"skill":7,"startup_fit":6,"ba...
    ✅ Contains valid JSON: ['skill', 'startup_fit', 'background', 'overall', 'summary', 'followup_tips']
-----------------------------

In [44]:
# Extract and format messages in a structured way
def extract_content_text(content_raw):
    """Extract text from content which may be a list of ResponseOutputText objects."""
    if isinstance(content_raw, list):
        text_parts = []
        for content_item in content_raw:
            if hasattr(content_item, 'text'):
                text_parts.append(content_item.text)
            elif isinstance(content_item, str):
                text_parts.append(content_item)
        return "".join(text_parts)
    elif isinstance(content_raw, str):
        return content_raw
    else:
        return str(content_raw)

messages = []
for item in items:
    if hasattr(item, 'type') and item.type == 'message':
        role = getattr(item, 'role', 'unknown')
        content_raw = getattr(item, 'content', '')
        content_text = extract_content_text(content_raw)
        item_id = getattr(item, 'id', '')
        
        messages.append({
            "id": item_id,
            "role": role,
            "content": content_text,
            "content_length": len(content_text)
        })

# Display summary
print(f"📊 Summary:")
print(f"   Total messages: {len(messages)}")
print(f"   Roles: {set(m['role'] for m in messages)}")
print(f"\n📝 Messages by role:")
for role in set(m['role'] for m in messages):
    role_messages = [m for m in messages if m['role'] == role]
    print(f"   {role}: {len(role_messages)} message(s)")


📊 Summary:
   Total messages: 10
   Roles: {'user', 'developer', 'assistant'}

📝 Messages by role:
   user: 3 message(s)
   developer: 4 message(s)
   assistant: 3 message(s)


In [45]:
# Display full messages in a readable format
print("\n" + "=" * 80)
print("FULL MESSAGE CONTENT")
print("=" * 80)

for idx, msg in enumerate(messages, 1):
    print(f"\n{'='*80}")
    print(f"Message #{idx} | Role: {msg['role']} | ID: {msg['id']}")
    print(f"{'='*80}")
    print(msg['content'])
    print(f"\n{'-'*80}")



FULL MESSAGE CONTENT

Message #1 | Role: developer | ID: msg_6941e3b17fc0819782daf81aec0d444f016d31c69598a7b7
你是招聘顾问，正在和候选人沟通。你的目标是根据岗位要求分析简历，            通过对话的方式判断候选人是否符合岗位要求，可以提出问题让候选人回答，也可以介            绍岗位要求。如果候选人的经验以及回复达到岗位要求，则可以推进到面试阶段，            表达会通知HR尽快安排面试。            以下是岗位描述，用于分析候选人的匹配程度:
{"updated_at": "2025-12-13T19:54:41.556892", "drill_down_questions": "(本章节包含 20 个没有标准答案的开放式问题。请 AI Agent 根据候选人简历中的技术栈，选择 1-3 个进行追问。重点评估候选人的思考路径，而非仅仅看结果。)\n\n5.1 维度一：架构决策与权衡 (Architecture Trade-offs)\n\n考察点：是否理解“架构即取舍”，能否在特定约束下做最优解，而非盲目上新技术。\n\nQ1. [自研 vs 开源] 工作流引擎的边界\n\n场景: \"我们需要实现一个类似 Airflow 的 DAG 调度系统，但核心要求是支持毫秒级延迟和状态热更新。业界有 Temporal, Cadence, Argo 等成熟方案。\n问题: 如果让你决策，你会直接引入 Temporal，还是基于 DB/Redis 自研一个轻量级状态机？请列出你做这个决策的决策树（Decision Tree），并告诉我什么情况下你会推翻自己的决定？\"\n\n✅ 期望回答: 能分析 Temporal 的运维成本 (Cassandra/ES 依赖) vs 自研的各种坑 (状态一致性、重试风暴)。提到“团队熟悉度”和“业务复杂度”作为变量。\n\n❌ 警惕: 直接说“Temporal 是业界标准所以用它”，或者“自研更可控”这种万能句式。\n\nQ2. [数据模型] JSONB 的爱与恨\n\n场景: \"我们的任务参数极其灵活，Schema 经常变。我们使用了 Postgres 的 JSONB。但随着数据量增长，

In [51]:
msg

{'id': 'msg_016d31c69598a7b700694386c783408197ba0259ebc8642f80',
 'role': 'assistant',
 'content': '{"word_count":64}',
 'content_length': 17}

In [46]:
# Export to JSON for further analysis
export_data = {
    "conversation_id": conversation_id,
    "retrieved_at": datetime.now().isoformat(),
    "conversation_metadata": getattr(conversation, 'metadata', {}),
    "total_items": len(items),
    "messages": messages,
    "has_more": getattr(items_response_meta, 'has_more', False),
    "all_items": [
        {
            "type": getattr(item, 'type', 'unknown'),
            "id": getattr(item, 'id', 'N/A'),
            "role": getattr(item, 'role', None) if hasattr(item, 'role') else None,
            "content": extract_content_text(getattr(item, 'content', '')) if hasattr(item, 'content') else '',
        }
        for item in items
    ]
}

# Save to file
output_file = Path.cwd() / f"conversation_{conversation_id[:20]}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"✅ Exported conversation data to: {output_file}")
print(f"   Total messages: {len(messages)}")
print(f"   File size: {output_file.stat().st_size} bytes")


✅ Exported conversation data to: /Users/derek/Documents/Projects/bosszhipin_bot/scripts/conversation_conv_6941e3b17fa0819.json
   Total messages: 10
   File size: 68724 bytes


In [53]:
instruction = """你作为星尘数据的招聘顾问，请用轻松口语化的风格和候选人沟通。请直接生成一条可以发送给候选人的自然语言消息，不要超过150字。不要发模板或者嵌入占位符，不要使用任何格式化、引号、JSON或括号。
候选人已经读了你的消息，但是有一段时间没有回复了，请根据之前的对话历史，生成下一条跟进消息，用于吸引候选人回复。
可以介绍公司情况、岗位亮点，根据岗位描述和这个候选人的特点，思考如何吸引候选人，并打消其顾虑、提高他的求职意愿，重点是让他对这个岗位有兴趣。"""

response = client.responses.create(
    conversation=conversation_id,
    # previous_response_id='msg_016d31c69598a7b7006941e3ef4504819782945129034613ba',
    instructions=instruction,
    input='[沉默]',
    model='gpt-5-mini',
    tools=[{"type": "web_search"}],  # Enable web search tool
)
response.output_text

'张宇你好，我是星尘数据的招聘顾问。你在医疗SaaS、微服务和私有化交付的实战很符合MorningStar现在的刚需——岗位会主导核心工作流引擎、断点续传和类Git数据血缘设计，影响面大、能产出代码与技术沉淀。要是你对Temporal或Postgres JSONB还没深度经验也没关系，我们更看你的架构思路和实操能力。方便简单回复上次那两题，或告诉我一个15分钟通话的时间？我可以帮你争取优先面试和薪资对接。'